# Импорт библиотек

In [1]:
import pandas as pd
from datetime import datetime

import json
import os

# Чтение данных

In [3]:
all_data = []
# Перебираем все файлы в директории
for filename in os.listdir('data'):
    if filename.endswith('.json'): # Проверяем, что файл имеет расширение .json
        file_path = os.path.join('data', filename)
        # Открываем
        with open(file_path, 'r', encoding='utf-8') as file:
            data = [json.loads(line) for line in file]
            all_data.extend(data)

In [4]:
data_df = pd.DataFrame(all_data)
# Так как в данных есть списки со словарями, преобразуем их просто в словари, чтобы можно было преобразить данные в DataFrame без словарей:
# Ключи словарей станут новыми признаками в DataFrame 
def convert_list_to_dict(cell):
    if isinstance(cell, list) and len(cell) > 0:
        return cell[0]
    else:
        return cell

# Применение функции к каждой ячейке в каждом столбце DataFrame
for column in data_df.columns:
    data_df[column] = data_df[column].apply(convert_list_to_dict)

In [5]:
# Преобразуем данные в json, чтобы открыть через json_normalize, сразу добавив ключи, как признаки
data_df_1 = pd.DataFrame(data_df)

clear_json = data_df_1.to_json(orient='records')

data_list = json.loads(clear_json)

full_data = pd.json_normalize(data_list)
# отсортируем признаки по алфавиту
full_data = full_data.sort_index(axis=1)

In [6]:
# Запишем в  готовую таблицу в csv файл
full_data.to_csv('data/Полная таблица.csv')


# Знакомство с данными

In [7]:
df = pd.read_csv('data/Полная таблица.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

C:\Users\Aplatonov\AppData\Local\Temp\ipykernel_3044\2871854370.py:1: DtypeWarning: Columns (171,175,177,179,182,201,202,203,204,205,206,211,214,215) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Полная таблица.csv')


,addOkved,addOkved.code,addOkved.name,addOkved.version,address,administrators,administrators.name,administrators.title,charter,dateLiquid,...,socialMedia.youtube,statusDetail.code,statusDetail.name,statusDetail.shortName,successors,successors.fullName,successors.inn,successors.ogrn,website,website_punycode
0,[],NaN,NaN,NaN,"191186, ГОРОД САНКТ-ПЕТЕРБУРГ, УЛИЦА ИТАЛЬЯНСК...",NaN,КОШЕЧКИН ДЕНИС ВИКТОРОВИЧ,ПРЕДСЕДАТЕЛЬ ПРАВЛЕНИЯ,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
1,NaN,58.11,Издание книг,ОК 029-2014 (КДЕС Ред. 2),"111395, ГОРОД МОСКВА, УЛИЦА МОЛДАГУЛОВОЙ, ДОМ ...",NaN,НИЗАМИЕВ РУСЛАН ВИЛЬДАНОВИЧ,ГЕНЕРАЛЬНЫЙ ДИРЕКТОР,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
2,[],NaN,NaN,NaN,"197198, ГОРОД САНКТ-ПЕТЕРБУРГ, УЛИЦА БОЛЬШАЯ П...",NaN,САМОЙЛОВА АНЖЕЛА ВАСИЛЬЕВНА,ПРЕДСЕДАТЕЛЬ КОЛЛЕГИИ,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
3,NaN,68.20,Аренда и управление собственным или арендованн...,ОК 029-2014 (КДЕС Ред. 2),"125424, ГОРОД МОСКВА, ШОССЕ ВОЛОКОЛАМСКОЕ, ДОМ...",NaN,ГЕЛЬДТ ВЛАДИМИР АНДРЕЕВИЧ,ПРЕДСЕДАТЕЛЬ ТОВАРИЩЕСТВА,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN
4,NaN,73.20,Исследование конъюнктуры рынка и изучение обще...,ОК 029-2014 (КДЕС Ред. 2),"197198, ГОРОД САНКТ-ПЕТЕРБУРГ, УЛИЦА ЗВЕРИНСКА...",NaN,ПРИВАЛОВА ЮЛИЯ АНАТОЛЬЕВНА,ДИРЕКТОР,NaN,NaN,...,NaN,NaN,NaN,Действующая,[],NaN,NaN,NaN,NaN,NaN


In [8]:
print(df.info(verbose=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 215 columns):
 #    Column                                Dtype  
---   ------                                -----  
 0    addOkved                              object 
 1    addOkved.code                         object 
 2    addOkved.name                         object 
 3    addOkved.version                      object 
 4    address                               object 
 5    administrators                        object 
 6    administrators.name                   object 
 7    administrators.title                  object 
 8    charter                               float64
 9    dateLiquid                            float64
 10   dateLiquid.$date                      object 
 11   dateOgrn.$date                        object 
 12   dateReg                               float64
 13   dateReg.$date                         object 
 14   egrulStatus                           object 
 15 

In [9]:
missing_percentage = df.isnull().mean() * 100
print(f'В данных {missing_percentage[missing_percentage == 0].count()} признаков без записей (только пропуски) и {missing_percentage[(missing_percentage != 0) & (missing_percentage <= 10)].count()} признаков, где до 10% записей')
# Признаки только с пропусками можно удалить
df_no_nan = df.dropna(axis=1, how='all')

В данных 21 признаков без записей (только пропуски) и 10 признаков, где до 10% записей


# Очистка данных

In [10]:
# Просмотрим все признаки типа object
object_columns = df.select_dtypes(include=['object'])
print(len(object_columns.columns), 'столбцов object')

47 столбцов object


In [11]:
df.shape

(747034, 215)

In [12]:
lest_obj = []
for col in object_columns.columns:
    if df[col].nunique() == 1: # выберем столбцы, в которых одина уникальная запись
        lest_obj.append(col)
        print(df[col].value_counts())

addOkved
[]    474366
Name: count, dtype: int64
administrators
[]    48722
Name: count, dtype: int64
predecessors
[]    735550
Name: count, dtype: int64
reports
[]    614158
Name: count, dtype: int64
socialMedia.instagram
https://www.instagram.com/crimea24news    1
Name: count, dtype: int64
successors
[]    732281
Name: count, dtype: int64


In [13]:
# Так как это просто пустые списки и одна ссылка на соцсеть, можно удалить эти признаки
df_no_nan = df.drop(columns=lest_obj)

In [15]:
df_no_nan.shape
# Количество кризнаков сократилось в четыре раза 

(747034, 54)

In [17]:
df_no_nan.to_csv('data/Полная таблица очищенная.csv')

### Проанализируем признаки

In [2]:
df_full = pd.read_csv('data/Полная таблица очищенная.csv')
df_full = df_full.copy()

C:\Users\Aplatonov\AppData\Local\Temp\ipykernel_16368\3928868559.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('data/Полная таблица очищенная.csv')


In [3]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747034 entries, 0 to 747033
Data columns (total 55 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            747034 non-null  int64  
 1   addOkved.code                         272668 non-null  object 
 2   addOkved.name                         272668 non-null  object 
 3   addOkved.version                      272668 non-null  object 
 4   address                               747034 non-null  object 
 5   administrators.name                   698312 non-null  object 
 6   administrators.title                  698312 non-null  object 
 7   dateLiquid.$date                      399961 non-null  object 
 8   dateOgrn.$date                        747034 non-null  object 
 9   dateReg.$date                         321416 non-null  object 
 10  egrulStatus                           747034 non-null  object 
 11  

- От дополнительного ОКВЭД вдяд ли есть зависимость одобрения гранта - удаляем
- Точный адрес тоже не должен влиять (посммотрим позже на регион и город) - удаляем
- Имя руководителя как и должность тоже ничего не должно значить - удаляем
- ИНН и КПП не влияют - удаляем
- Код основного ОКВЭД и  его версия ничего нам не скажет (оставиим название) - удаляем
- Реестровый номер НКО в системе Минюста РФ - удаляем
- ОГРН - удаляем
- код ОКАТО точно не нужен, оставим расшифровку
- код ОКФС - удаляем
- код ОКОГУ - удаляем
- код ОКТМО - удаляем
- код ОПФ и версия- удаляем
- ID отчета и тип - удаляем

- Статус НКО в системе Минюста РФ не должен влиять на одобрение гранта, но пока оставим

Остальные признаки будем обрабатывать.   
Возможно, ещё что-то удалим в процессе

hasRegionalSupport - целевой признак одобрения получения региональной поддержки

In [4]:
del_list = ['Unnamed: 0',
    'addOkved.code',
    'addOkved.version',
    'address',
    'administrators.name',
    'administrators.title',
    'inn',
    'kpp',
    'mainOkved.code',
    'mainOkved.version',
    'ogrn',
    'okato.code',
    'okfs.code',
    'okogu.code',
    'oktmo.code',
    'opf.code',
    'opf.version',
    'reports.reportId',
    'reports.type'
]
df_clean = df_full.drop(columns=del_list)

In [5]:
# Удалим дубликаты и пустые строки
df_clean_dub = df_clean.drop_duplicates() 
df_clean_nan = df_clean_dub.dropna(thresh = int(len(df_clean_dub.columns))*0.2) # удаляем строки, где более 80% пропусков в записи 
print('Количество дубликатов: ', df_clean.shape[0]-df_clean_dub.shape[0])
print('Количество пустых строк: ', df_clean_dub.shape[0]-df_clean_nan.shape[0])

Количество дубликатов:  26
Количество пустых строк:  0


# Предварителньный анализ

Регион регистрации

In [6]:
df_clean_nan['regionName'].value_counts()

regionName
Москва                          80175
Московская область              37924
Санкт-Петербург                 31766
Краснодарский край              23337
Свердловская область            23028
                                ...  
Республика Ингушетия             1248
Еврейская автономная область     1197
Чукотский автономный округ        505
Ненецкий автономный округ         398
Не определено                      91
Name: count, Length: 86, dtype: int64

Логично, что больше всего НКО в самых населённых регионах

In [7]:
region_group = df_clean_nan.pivot_table(index = 'regionName', columns='hasRegionalSupport', aggfunc='size', fill_value=0)
region_group

hasRegionalSupport,False,True
regionName,,
Алтайский край,10585,316
Амурская область,2863,28
Архангельская область,5637,177
Астраханская область,4232,9
Белгородская область,7945,46
...,...,...
Чеченская республика,1794,15
Чувашская республика,5306,121
Чукотский автономный округ,457,48


In [8]:
region_group['%_approve'] = round((region_group[True] / region_group.sum(axis=1)) * 100)
region_group.sort_values('%_approve', ascending=False).head(10)

hasRegionalSupport,False,True,%_approve
regionName,,,
Ненецкий автономный округ,306,92,23.0
Чукотский автономный округ,457,48,10.0
Ямало-Ненецкий автономный округ,2515,248,9.0
Магаданская область,1456,94,6.0
Псковская область,3052,165,5.0
Республика Калмыкия,1664,89,5.0
Сахалинская область,3649,181,5.0
Калининградская область,6585,319,5.0
Кабардино-Балкарская республика,2406,117,5.0


Больше всего процент одобрений в НАО.

# Анализ зависимости от времени 

In [16]:
df_clean_nan['originDate.$date'] = pd.to_datetime(df_clean_nan['originDate.$date'])
df_clean_nan['originDate.$date'].sort_values().head(10)

37554    1881-01-29 00:00:00+00:00
417948   1900-01-16 00:00:00+00:00
742634   1900-07-05 00:00:00+00:00
423917   1912-01-28 00:00:00+00:00
606649   1922-08-21 00:00:00+00:00
687624   1933-04-15 00:00:00+00:00
237878   1933-07-01 00:00:00+00:00
487425   1933-07-14 00:00:00+00:00
640245   1936-05-07 00:00:00+00:00
303284   1939-02-01 00:00:00+00:00
Name: originDate.$date, dtype: datetime64[ns, UTC]

In [19]:
# Проверим самые старые организации
df_clean_nan[df_clean_nan['originDate.$date'] <= '1940']
# Записи выглядят нормальными, можно оставить

,addOkved.name,dateLiquid.$date,dateOgrn.$date,dateReg.$date,egrulStatus,fullName,hasRegionalSupport,incomeDetail.contracts223.totalCount,incomeDetail.contracts223.totalSum,incomeDetail.contracts44.totalCount,...,originDate.$date,regionCode,regionName,reports.localUrl,reports.minjustUrl,reports.publishYear,shortName,statusDetail.shortName,Month_s,Year_s
37554,NaN,2019-08-27T00:00:00.000Z,2002-07-26T00:00:00.000Z,1881-01-29T00:00:00.000Z,Ликвидирована,"САДОВОДЧЕСКОЕ НЕКОММЕРЧЕСКОЕ ТОВАРИЩЕСТВО ""ПРО...",False,0,0.0,0,...,1881-01-29 00:00:00+00:00,50,Московская область,NaN,NaN,NaN,"СНТ ""ПРОЛЕТАРЕЦ-4""",Действующая,1719,143
237878,NaN,2007-05-11T00:00:00.000Z,2007-05-11T00:00:00.000Z,1933-07-01T00:00:00.000Z,Ликвидирована,"КООПЕРАТИВ ""ПОЛИСЕРВИС ЦЕНТР""",False,0,0.0,0,...,1933-07-01 00:00:00+00:00,27,Хабаровский край,NaN,NaN,NaN,"КООПЕРАТИВ ""ПОЛИСЕРВИС ЦЕНТР""",Действующая,1089,91
303284,NaN,2011-10-27T00:00:00.000Z,2011-10-27T00:00:00.000Z,1939-02-01T00:00:00.000Z,Ликвидирована,"КООПЕРАТИВ ""ГЕРМЕС""",False,0,0.0,0,...,1939-02-01 00:00:00+00:00,29,Архангельская область,NaN,NaN,NaN,"КООПЕРАТИВ ""ГЕРМЕС""",Действующая,1022,85
417948,NaN,2008-07-03T00:00:00.000Z,2008-07-03T00:00:00.000Z,1900-01-16T00:00:00.000Z,Ликвидирована,РЕМОНТНО-РЕСТАВРАЦИОННЫЙ СТРОИТЕЛЬНЫЙ КООПЕРАТ...,False,0,0.0,0,...,1900-01-16 00:00:00+00:00,63,Самарская область,NaN,NaN,NaN,NaN,Действующая,1491,124
423917,NaN,2008-05-06T00:00:00.000Z,2008-05-06T00:00:00.000Z,1912-01-28T00:00:00.000Z,Ликвидирована,ТВОРЧЕСКАЯ МАСТЕРСКАЯ АРХИТЕКТОРА Н.В.БЕЛОУСОВА,False,0,0.0,0,...,1912-01-28 00:00:00+00:00,77,Москва,NaN,NaN,NaN,ТВОРЧЕСКАЯ МАСТЕРСКАЯ АРХИТЕКТОРА Н.В.БЕЛОУСОВА,Действующая,1347,112
487425,NaN,NaN,2003-04-17T00:00:00.000Z,1933-07-14T00:00:00.000Z,Ликвидирована,ВОЙСКОВАЯ ЧАСТЬ 2495,False,0,0.0,0,...,1933-07-14 00:00:00+00:00,79,Еврейская автономная область,NaN,NaN,NaN,В/Ч 2495,Недействующая,1089,91
606649,NaN,NaN,2002-10-22T00:00:00.000Z,1922-08-21T00:00:00.000Z,Действует,АРХАНГЕЛЬСКАЯ ОБЛАСТНАЯ КОЛЛЕГИЯ АДВОКАТОВ,False,0,0.0,0,...,1922-08-21 00:00:00+00:00,29,Архангельская область,"http://cdnx.h5.crftr.net/f/4,0b81e2b07ceccf.pdf",http://unro.minjust.ru/NKOReports.aspx?mode=sh...,2012.0,АОКА,Действующая,1220,102
640245,NaN,2004-05-12T00:00:00.000Z,2002-12-20T00:00:00.000Z,1936-05-07T00:00:00.000Z,Ликвидирована,ВОЙСКОВАЯ ЧАСТЬ 9783,False,0,0.0,0,...,1936-05-07 00:00:00+00:00,27,Хабаровский край,NaN,NaN,NaN,В/Ч 9783,Действующая,1055,88
687624,NaN,2017-12-15T00:00:00.000Z,2017-12-15T00:00:00.000Z,1933-04-15T00:00:00.000Z,Ликвидирована,ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ ПРОФСОЮЗНЫЙ КОМИТЕТ МУ...,False,0,0.0,0,...,1933-04-15 00:00:00+00:00,23,Краснодарский край,NaN,NaN,NaN,NaN,Действующая,1092,91
742634,NaN,2013-10-21T00:00:00.000Z,2006-10-05T00:00:00.000Z,1900-07-05T00:00:00.000Z,Ликвидирована,"ГАРАЖНЫЙ ПОТРЕБИТЕЛЬСКИЙ КООПЕРАТИВ ""ГАРАЖНЫЙ ...",False,0,0.0,0,...,1900-07-05 00:00:00+00:00,59,Пермский край,NaN,NaN,NaN,ГПК №16,Действующая,1485,124


In [12]:
# Добавим признак Количество месяцев, чтобы проверить, влияет ли "возраст" компании на одобрямость
df_clean_nan['originDate.$date'] = pd.to_datetime(df_clean_nan['originDate.$date'])
today = datetime.now()
df_clean_nan['Month_s'] = ((today.year - df_clean_nan['originDate.$date'].dt.year) * 12 + today.month - df_clean_nan['originDate.$date'].dt.month)
df_clean_nan['Year_s'] = today.year - df_clean_nan['originDate.$date'].dt.year

In [22]:
time_group = df_clean_nan.pivot_table(index = 'Year_s', columns='hasRegionalSupport', aggfunc='size', fill_value=0)

time_group['Year_%'] = round((time_group[True] / time_group.sum(axis=1)) * 100)
time_group.sort_values('Year_%', ascending=False).head(20)

hasRegionalSupport,False,True,Year_%
Year_s,,,
78,2,1,33.0
76,7,1,12.0
11,19532,753,4.0
13,20178,755,4.0
9,21964,578,3.0
10,17860,602,3.0
12,21365,731,3.0
14,23293,643,3.0
16,28262,452,2.0


Больше всего одобрений у компаний, которые открылись 78 и 76 лет назад, но в те года в принципе было мало регистраций.  
Таким образом сложно сказать о зависимости одобряемости от возраста компании.  
Скорее всего это ошибочные данные. Проверим их.

In [23]:
df_clean_nan[(df_clean_nan['Year_s'] == 78) | (df_clean_nan['Year_s'] == 76)]
# Данные вполне обычные. Не будем их исключать.

,addOkved.name,dateLiquid.$date,dateOgrn.$date,dateReg.$date,egrulStatus,fullName,hasRegionalSupport,incomeDetail.contracts223.totalCount,incomeDetail.contracts223.totalSum,incomeDetail.contracts44.totalCount,...,originDate.$date,regionCode,regionName,reports.localUrl,reports.minjustUrl,reports.publishYear,shortName,statusDetail.shortName,Month_s,Year_s
65643,NaN,2020-08-12T00:00:00.000Z,2003-01-08T00:00:00.000Z,1946-05-13T00:00:00.000Z,Ликвидирована,ВОЙСКОВАЯ ЧАСТЬ 15644,False,0,0.0,0,...,1946-05-13 00:00:00+00:00,30,Астраханская область,NaN,NaN,NaN,NaN,Действующая,935,78
99191,NaN,NaN,2003-02-14T00:00:00.000Z,1948-09-25T00:00:00.000Z,Действует,"ТОВАРИЩЕСТВО СОБСТВЕННИКОВ НЕДВИЖИМОСТИ ""ПЧЕЛК...",False,0,0.0,0,...,1948-09-25 00:00:00+00:00,64,Саратовская область,NaN,NaN,NaN,"ТСН ""ПЧЕЛКА-I""",Действующая,907,76
152528,Аренда и управление собственным или арендованн...,NaN,2002-12-18T00:00:00.000Z,1948-10-30T00:00:00.000Z,Действует,"СОЮЗ ""КРАСНОДАРСКОЕ КРАЕВОЕ ОБЪЕДИНЕНИЕ ОРГАНИ...",True,1,180000.0,22,...,1948-10-30 00:00:00+00:00,23,Краснодарский край,NaN,NaN,NaN,КРАСНОДАРСКОЕ КРАЕВОЕ ПРОФОБЪЕДИНЕНИЕ,Действующая,906,76
260754,NaN,2018-06-25T00:00:00.000Z,2018-06-25T00:00:00.000Z,1948-10-25T00:00:00.000Z,Ликвидирована,ПРОФСОЮЗНЫЙ КОМИТЕТ ДЕТСКОЙ МУНИЦИПАЛЬНОЙ КЛИН...,False,0,0.0,0,...,1948-10-25 00:00:00+00:00,27,Хабаровский край,NaN,NaN,NaN,ПРОФКОМ ДМК БОЛЬНИЦЫ ИМ.ИСТОМИНА /ОБ,Действующая,906,76
329397,NaN,2012-05-10T00:00:00.000Z,2012-05-10T00:00:00.000Z,1946-06-29T00:00:00.000Z,Ликвидирована,ПРОФСОЮЗНЫЙ КОМИТЕТ ПРЕПОДАВАТЕЛЕЙ МЕДИЦИНСКОГ...,False,0,0.0,0,...,1946-06-29 00:00:00+00:00,45,Курганская область,NaN,NaN,NaN,ПРОФСОЮЗНЫЙ КОМИТЕТ ПРЕПОДАВАТЕЛЕЙ МЕДИЦИНСКОГ...,Действующая,934,78
330139,NaN,2012-04-05T00:00:00.000Z,2012-04-05T00:00:00.000Z,1948-09-27T00:00:00.000Z,Ликвидирована,ПРОФСОЮЗНЫЙ КОМИТЕТ АО КУРГАНТРУБОПРОВОДСТРОЙ,False,0,0.0,0,...,1948-09-27 00:00:00+00:00,45,Курганская область,NaN,NaN,NaN,ПРОФКОМ АО КУРГАНТРУБОПРОВОДСТРОЙ,Действующая,907,76
330830,NaN,2012-04-05T00:00:00.000Z,2012-04-05T00:00:00.000Z,1948-09-27T00:00:00.000Z,Ликвидирована,ПРОФСОЮЗНЫЙ КОМИТЕТ ЦЕНТРА СТАНДАРТИЗАЦИИ МЕТР...,False,0,0.0,0,...,1948-09-27 00:00:00+00:00,45,Курганская область,NaN,NaN,NaN,ПРОФКОМ ЦЕНТРА СТАНДАРТИЗАЦИИ И МЕТРОЛОГИИ,Действующая,907,76
355273,NaN,NaN,2002-10-09T00:00:00.000Z,1948-04-26T00:00:00.000Z,Действует,"ДАЧНЫЙ ПОТРЕБИТЕЛЬСКИЙ КООПЕРАТИВ ""СОЛНЕЧНОЕ""",False,0,0.0,0,...,1948-04-26 00:00:00+00:00,50,Московская область,NaN,NaN,NaN,"""ДК ""СОЛНЕЧНОЕ""",Действующая,912,76
396928,NaN,NaN,2002-12-19T00:00:00.000Z,1946-05-15T00:00:00.000Z,Действует,"НЕГОСУДАРСТВЕННАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""...",True,0,0.0,0,...,1946-05-15 00:00:00+00:00,39,Калининградская область,"http://cdnx.h5.crftr.net/f/6,0b3374d001d1f5.pdf",http://unro.minjust.ru/NKOReports.aspx?mode=sh...,2011.0,"ННО ""КОКА""",Действующая,935,78
571168,NaN,2014-06-05T00:00:00.000Z,2014-06-05T00:00:00.000Z,1948-09-27T00:00:00.000Z,Ликвидирована,ПРОФСОЮЗНЫЙ КОМИТЕТ АО КУРГАНТРАНСМАШПРОЕКТ,False,0,0.0,0,...,1948-09-27 00:00:00+00:00,45,Курганская область,NaN,NaN,NaN,ПРОФКОМ АО КУРГАНТРАНСМАШПРОЕКТ,Действующая,907,76


In [ ]:
region_group['Year_s'] = round((region_group[True] / region_group.sum(axis=1)) * 100)
region_group.sort_values('%_approve', ascending=False).head(10)

In [30]:
# Вычислим процент Nan-строк в каждом столбце
is_nan = pd.DataFrame()
for column in df_clean_nan.columns:
    num_nan = df_clean_nan[column].isna().sum()
    percentage = (num_nan / len(df_clean_nan)) * 100
    is_nan[column] = [percentage]

display(is_nan)

,addOkved.name,dateLiquid.$date,dateOgrn.$date,dateReg.$date,egrulStatus,fullName,hasRegionalSupport,incomeDetail.contracts223.totalCount,incomeDetail.contracts223.totalSum,incomeDetail.contracts44.totalCount,...,originDate.$date,regionCode,regionName,reports.localUrl,reports.minjustUrl,reports.publishYear,shortName,statusDetail.shortName,Month_s,Year_s
0,63.498918,46.460547,0.0,56.97516,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.003347,82.217192,82.212239,82.212239,19.976761,0.0,0.0,0.0


In [42]:
not_0 = is_nan[is_nan >= 1]
# not_0 = not_0[not_0 == True]
not_0 = not_0.dropna(axis=1)
not_0

,addOkved.name,dateLiquid.$date,dateReg.$date,mainOkved.name,minjustForm,minjustRegNum,minjustStatus,okato.name,okfs.name,okogu.name,okpo,oktmo.name,reports.localUrl,reports.minjustUrl,reports.publishYear,shortName
0,63.498918,46.460547,56.97516,27.397163,38.277368,38.144304,38.112845,54.723376,53.489521,53.490993,53.489521,63.090891,82.217192,82.212239,82.212239,19.976761


Точно можно удалить признаки об отчётах в Минюст РФ. Скорее всего, они не несут важной информации. К тому же в них больше 80% пропусков.

In [26]:
# Количество строк с пустыми признаками
rows_with_nan = df_clean_nan.isna().any(axis=1).sum()

print(df_clean_nan.shape[0]-rows_with_nan, 'строк без пропусков')

183 строк без пропусков
